# Compute spherical harmonics

Make only  
    # 'shape_mode_1_height', 'shape_mode_2_volume',  
    # 'shape_mode_3_major_tilt', 'shape_mode_4_minor_tilt',  
    # 'shape_mode_5_elongation', 'shape_mode_6_bean-ness',  
    # 'shape_mode_7_pear-ness', 'shape_mode_8_wedge',  
Use cell segmentation channel, and `compute_lcc = False` to speed-up computation

In [28]:
# read_ome_zarr?

In [27]:
# !cat /opt/conda/lib/python3.10/site-packages/aicsimageprocessing/diagnosticsheet.py

In [ ]:
# If not already installed, run:
# !pip install aicsshparam

In [11]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from aicsimageprocessing import diagnostic_sheet, read_ome_zarr, rescale_image, imgtoprojection
from upath import UPath as Path
import warnings
from aicsimageio import transforms, AICSImage
import nbvv
from sklearn.decomposition import PCA
from aicsshparam import shtools, shparam
from time import time
from tqdm import trange

In [2]:
df = pd.read_csv("../data/mitocells.csv")
df.set_index(df['CellId'].astype(int), inplace=True)
print(f'Number of cells: {len(df)}')
print(f'Number of columns: {len(df.columns)}')

Number of cells: 10381
Number of columns: 79


In [3]:
col_df = pd.read_csv("resources/hackathon_column_descriptions.csv",delimiter=",")
col_df.columns = ["column name","description","category"]
# for index, row in col_df.iterrows():
#     print(row.values)

In [6]:
some_cell = df.sample(1).iloc[0]

df_coeffs = pd.DataFrame([])
for level in [0, 1, 2]:
    # Final images saved under column name "3d_image"
    full_img = read_ome_zarr(some_cell["3d_image"], level=level)
    if level == 0:
        # Inspect whether this image fits the bounding box in z, y, and x
        print(some_cell[[i for i in some_cell.index if "fit" in i]])
        # Images have 7 channels for brightfield + DNA, membrane, and structure intensity and segmentation channels
        print(full_img.channel_names)
    img = full_img.data.squeeze()[5]
    print(img.shape)
    plt.figure()
    plt.imshow(img.max(0), aspect='auto')
    plt.colorbar()
    plt.show()
    (coeffs, _), _ = shparam.get_shcoeffs(image=img, lmax=16)
    df_coeffs = pd.concat([df_coeffs, pd.Series(coeffs)], axis=1, ignore_index=True)
print(df_coeffs)

fits_x    True
fits_y    True
fits_z    True
Name: 548948, dtype: object
['bf', 'dna', 'membrane', 'structure', 'dna_segmentation', 'membrane_segmentation', 'struct_segmentation_roof']
(129, 238, 374)
(64, 119, 187)
(32, 60, 94)
                          0          1          2
shcoeffs_L0M0C    76.638496  37.802534  18.748446
shcoeffs_L0M1C     0.000000   0.000000   0.000000
shcoeffs_L0M2C     0.000000   0.000000   0.000000
shcoeffs_L0M3C     0.000000   0.000000   0.000000
shcoeffs_L0M4C     0.000000   0.000000   0.000000
...                     ...        ...        ...
shcoeffs_L16M12S   0.051115   0.026313   0.017177
shcoeffs_L16M13S  -0.089246  -0.046740  -0.015847
shcoeffs_L16M14S  -0.098975  -0.071369  -0.022108
shcoeffs_L16M15S  -0.005448  -0.003924   0.012508
shcoeffs_L16M16S   0.166166   0.083163   0.043693

[578 rows x 3 columns]


In [6]:
# Compute spherical harmonics coefficients of shape and store them
# in a pandas dataframe.
nb_imgs = 10
df_coeffs = pd.DataFrame([])
tic = time()
for i in range(nb_imgs):
    # load image
    some_cell = df.iloc[i, :]
    img = read_ome_zarr(some_cell["3d_image"], level=0).data.squeeze()[5]
    
    # Parameterize with L=4, which corresponds to 50 coefficients
    # in total
    (coeffs, _), _ = shparam.get_shcoeffs(image=img, lmax=4)
    coeffs.update({'CellId': df.index[i]})
    
    df_coeffs = pd.concat([df_coeffs, pd.Series(coeffs)], axis=1, ignore_index=True)
duration = time() - tic
print(f"Computation took {duration}s for {nb_imgs} images")


Computation took 65.8283588886261s for 10 images


In [76]:
res = []
for i in range(4, 17):
    (coeffs, _), _ = shparam.get_shcoeffs(image=img, lmax=i)
    res.append([i, len(coeffs)])
print(res)

[[4, 50], [5, 72], [6, 98], [7, 128], [8, 162], [9, 200], [10, 242], [11, 288], [12, 338], [13, 392], [14, 450], [15, 512], [16, 578]]


In [ ]:
level = 0
cell_ids = np.array(df.index.values) # copy values
np.random.shuffle(cell_ids)

df_coeffs = pd.DataFrame([])
tic = time()
for i in trange(len(cell_ids)):
# for i in trange(10):
    # load image
    cell_id = cell_ids[0]
    some_cell = df.loc[cell_id, :]
    img = read_ome_zarr(some_cell["3d_image"], level=level).data.squeeze()[5]
    
    # Parameterize with L=4, which corresponds to 50 coefficients
    # in total
    (coeffs, _), _ = shparam.get_shcoeffs(image=img, lmax=16)
    coeffs.update({'CellId': cell_id})
    cell_ids = cell_ids[1:]
    
    df_coeffs = pd.concat([df_coeffs, pd.Series(coeffs)], axis=1, ignore_index=False)
    if (i+1) % 100 == 0:
        df_coeffs.T.to_csv("../data/shape_coefficients.csv", index=False)
duration = time() - tic
print(f"Computation took {duration}s for {len(df)} images")

  0%|          | 31/10381 [03:36<20:09:51,  7.01s/it]/opt/conda/lib/python3.10/site-packages/aicsshparam/shparam.py:127: UserWarning: Mesh centroid seems to fall outside the object. This indicates        the mesh may not be a manifold suitable for spherical harmonics        parameterization.
  warnings.warn(
  0%|          | 48/10381 [05:33<19:51:56,  6.92s/it]